In [4]:
import pybroker as pyb
from datetime import datetime, timedelta
from pybroker import ExecContext, Strategy, YFinance, StrategyConfig
import talib
import pandas as pd

# Enable data source cache
pyb.enable_data_source_cache('rebalancing')

# Define the strategy configuration
config = StrategyConfig(initial_cash=100_000)

# Define helper functions

def cmr(df, w):
    """
    Calculate the cumulative return of stock data over a specified window.
    
    Parameters:
    - df: DataFrame containing the stock data, with 'close' column representing the closing prices.
    - w: Window size for calculating the cumulative return.
    
    Returns:
    - DataFrame with 'date' and 'cmr' columns representing the cumulative return values.
    """
    df['cmr'] = (1 + df['close'].pct_change()).rolling(window=w).apply(lambda x: x.prod(), raw=True) - 1
    return df[['date', 'cmr']]

def max_drawdown(df, w):
    """
    Calculate the maximum drawdown of stock data over a specified window.
    
    Parameters:
    - df: DataFrame containing the stock data, with 'close' column representing the closing prices.
    - w: Window size for calculating the maximum drawdown.
    
    Returns:
    - DataFrame with 'date' and 'max_drawdown' columns representing the maximum drawdown values.
    """
    rolling_max = df['close'].rolling(window=w).max()
    drawdown = (df['close'] - rolling_max) / rolling_max
    max_drawdown = drawdown.rolling(window=w).min()
    return pd.DataFrame({'date': df['date'], 'max_drawdown': max_drawdown})

# Define the strategy logic

def buy_low(ctxs: dict[str, ExecContext]):
    dt = tuple(ctxs.values())[0].dt
    date_object = datetime.strptime(str(dt), "%Y-%m-%d %H:%M:%S")
    dt2 = date_object.strftime("%Y-%m-%d")

    cur_vixm_rsi = vixm_rsi.loc[dt2]
    cur_bnd_cmr = bnd_cmr.loc[dt2].cmr
    cur_bil_cmr = bil_cmr.loc[dt2].cmr.values[0]
    cur_tlt_cmr_20 = tlt_cmr_20.loc[dt2].cmr
    cur_bil_cmr_20 = bil_cmr_20.loc[dt2].cmr
    cur_spy_draw_10 = spy_draw_10.loc[dt2]
    cur_sqqq_rsi_20 = sqqq_rsi_20.loc[dt2]
    cur_tbf_rsi_20 = tbf_rsi_20.loc[dt2]
    
    expected_alloc = {}
    
    if cur_vixm_rsi > 69:
        expected_alloc['SHY'] = 1
    else:
        if cur_bnd_cmr > cur_bil_cmr:
            risk_on = ["TECL", "SOXL", "FAS", "TQQQ", "UPRO", "TMF"]
            rsis_here = {}
            for r in risk_on:
                rsi = rsis_10[r].loc[dt2].values[0]
                rsis_here[r] = rsi
            rsis_here = dict(sorted(rsis_here.items(), key=lambda x: x[1]))
            to_buy = list(rsis_here.keys())[:3]
            for t in to_buy:
                expected_alloc[t] = 0.33
        else:
            if cur_tlt_cmr_20 < cur_bil_cmr_20:
                expected_alloc['USDU'] = 0.5
                if cur_sqqq_rsi_20 < cur_tbf_rsi_20:
                    expected_alloc['SQQQ'] = 0.5
                else:
                    expected_alloc['TBF'] = 0.5
            else:
                if abs(cur_spy_draw_10) < 0.05:
                    expected_alloc['UPRO'] = 0.55
                    expected_alloc['TMF'] = 0.45
                else:
                    expected_alloc['IEI'] = 0.25
                    expected_alloc['GLD'] = 0.25
                    expected_alloc['TIP'] = 0.25
                    expected_alloc['BSV'] = 0.25
    
    alloc[dt2] = expected_alloc

    for symbol, target_alloc in expected_alloc.items():
        ctx = ctxs[symbol]
        target_shares = ctx.calc_target_shares(target_alloc)
        pos = ctx.long_pos()

        if not pos:
            ctx.buy_shares = target_shares
        elif pos.shares < target_shares:
            ctx.buy_shares = target_shares - pos.shares
        elif pos.shares > target_shares:
            ctx.sell_shares = pos.shares - target_shares

# Define the main execution function

def shell(ctxs: dict[str, ExecContext]):
    pass

# Define the strategy and backtest

start_date = "01/01/2020"
end_date = "01/01/2021"

yf = YFinance()
start_date_temp = datetime.strptime(start_date, "%m/%d/%Y")

# Calculate the start date for fetching additional data
vixm_start_date = start_date_temp - timedelta(days=81)
bnd_start_date = start_date_temp - timedelta(days=120)
vixm_start_date = vixm_start_date.strftime("%m/%d/%Y")
bnd_start_date = bnd_start_date.strftime("%m/%d/%Y")

# Fetch initial data
vixm = yf.query("VIXM", vixm_start_date, end_date)
bnd = yf.query("BND", bnd_start_date, end_date)
bil = yf.query("BIL", bnd_start_date, end_date)
tlt = yf.query("TLT", bnd_start_date, end_date)
sqqq = yf.query("SQQQ", bnd_start_date, end_date)
tbf = yf.query("TBF", bnd_start_date, end_date)
spy = yf.query("SPY", bnd_start_date, end_date)

# Calculate indicators
rsis_10 = {}
for r in ["TECL", "SOXL", "FAS", "TQQQ", "UPRO", "TMF"]:
    d = yf.query(r, bnd_start_date, end_date)
    d1 = rsi_10(d)
    rsis_10[r] = d1

vixm_rsi = rsi_40(vixm)
bnd_cmr = cmr(bnd, 60)
bil_cmr = cmr(bil, 60)
tlt_cmr_20 = cmr(tlt, 20)
bil_cmr_20 = cmr(bil, 20)
sqqq_rsi_20 = rsi_20(sqqq)
tbf_rsi_20 = rsi_20(tbf)
spy_draw_10 = max_drawdown(spy, 10)

# Initialize allocation dictionary
alloc = {}

# Define and run the strategy
strategy = Strategy(YFinance(), start_date=start_date, end_date=end_date, config=config)
strategy.set_before_exec(buy_low)
strategy.add_execution(shell, ['SHY', 'TECL', 'SOXL', 'FAS', 'TQQQ', 'UPRO', 'TMF', 'USDU', 'SQQQ', 'TBF', 'IEI', 'GLD', 'TIP', 'BSV'])
result = strategy.backtest()


Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Loaded cached bar data.

Backtesting: 2020-01-01 00:00:00 to 2021-01-01 00:00:00

Loaded cached bar data.

Test split: 2020-01-02 00:00:00 to 2020-12-31 00:00:00


  0% (0 of 253) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--


KeyError: '2020-01-02'